In [1]:
! pip3 install requests
! pip3 install beautifulsoup4
import requests as rq
from bs4 import BeautifulSoup
import re

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
list_word = ['lanky', 'scrawny', 'bony', 'skinny'] # ДОБАВЛЯЕМ СЛОВА СПИСКОМ
count = 0
url_mtran_main_1 = 'https://www.multitran.com/m.exe?l1=1&l2=2&s='
url_mtran_main_2 = '&langlist=2'
url_mwd_main_1 = 'https://www.merriam-webster.com/dictionary/'
url_mwth_main_1 = 'https://www.merriam-webster.com/thesaurus/'
url_ety_main_1 = 'https://www.etymonline.com/word/'
url_ety_main_2 = '#etymonline_v_57'

with open('word_info.txt', 'w', encoding = 'utf-8') as word_info: # создаем новый док
    word_info.write('Word list')

for item in list_word:
    count += 1

# multitran   
    url_mtran_link = url_mtran_main_1 + item + url_mtran_main_2
    page_mtran = rq.get(url_mtran_link)
    page_mtran.encoding = 'utf-8'
    soup_mtran = BeautifulSoup(page_mtran.text, 'html.parser') 
    ru_words_mtran_concat = str()
# merriam-webster
    url_mwd_link = url_mwd_main_1 + item
    page_mwd = rq.get(url_mwd_link)
    page_mwd.encoding = 'utf-8'
    soup_mwd = BeautifulSoup(page_mwd.text, 'html.parser') 
    definition_mwd_concat = str()
# thesaurus
    url_mwth_link = url_mwth_main_1 + item
    page_mwth = rq.get(url_mwth_link)
    page_mwth.encoding = 'utf-8'
    soup_mwth = BeautifulSoup(page_mwth.text, 'html.parser') 
    syn_mwth_concat = str()
# etymonline
    url_ety_link = url_ety_main_1 + item + url_ety_main_2
    page_ety = rq.get(url_ety_link)
    page_ety.encoding = 'utf-8'
    soup_ety = BeautifulSoup(page_ety.text, 'html.parser')
    history_ety_concat = str()
    
    for words_mtran in soup_mtran.find_all('td', class_='trans'):
        words_mtran = words_mtran.text.lower()
        words_mtran = re.sub(r'\s\(.*?\)', '', words_mtran)   # убираем все, что в скобках
        words_mtran = re.sub(r'([a-z])+?-', '', words_mtran)  # убираем все, что написано латиницей с -
        words_mtran = re.sub(r'([a-z])+?', '', words_mtran)   # убираем все, что написано латиницей
        words_mtran = re.sub(r'\s\s+?', ')', words_mtran)     # убираем больше двух пробелов
        words_mtran = re.sub(r'\)*?\(*?,*?', '', words_mtran) # убираем отдельные скобки
        words_mtran = re.sub(r'$', ';', words_mtran)          # в конце строки ставим ;  
        words_mtran = re.sub(r'\s;', ';', words_mtran)        # убираем пробел перед ;
        words_mtran = re.sub(r';\s', ';', words_mtran)        # убираем пробел после ;
        words_mtran = words_mtran.strip()                     # убираем пробелы в начале и конце строки
        ru_words_mtran_concat = ru_words_mtran_concat + words_mtran
        ru_words_mtran_concat = re.sub(r';;', ';', ru_words_mtran_concat)
        ru_words_mtran_concat = re.sub(r'-*?\d*?\)*?\(*?,*?\'*?:*?\.*?', '', ru_words_mtran_concat)
        ru_words_mtran_concat = re.sub(r'\s\s+?', '', ru_words_mtran_concat) # убираем больше двух пробелов
        ru_words_mtran_list = list(filter(None, sorted(list(set(re.split(';', ru_words_mtran_concat))))))  
    main_header = str(count) + ' English word: ' + item.upper()
    translation_header = 'TRANSLATION: ' + url_mtran_link
    
    for definition_mwd in soup_mwd.find_all('span', class_='dtText'):
        definition_mwd = definition_mwd.text.lower()
        definition_mwd = re.sub(r'^\w+?\s*?:', ':', definition_mwd) # в начале стороки убираем все, что до :
        definition_mwd = re.sub(r'\s\s+?', '(', definition_mwd)     # больше 2х пробелов меняем на скобку
        definition_mwd = re.sub(r'\(.*?$', '', definition_mwd)      # убираем скобку и все знаки после до конца строки 
        definition_mwd = re.sub(r'^:\s', '', definition_mwd)        # убираем : и пробел в начале строки
        definition_mwd_concat = definition_mwd_concat + definition_mwd + '; '
    definition_header = 'DEFINITION: ' + url_mwd_link

    for syn_mwth in soup_mwth.find_all('span', class_='thes-list syn-list'):
        syn_mwth = syn_mwth.text.lower()
        syn_mwth = re.sub(r'\(.*?\)', '', syn_mwth)            # убираем все, что в скобках
        syn_mwth = re.sub(r'\[.*?\]', '', syn_mwth)            # убираем все, что в кв скобках
        syn_mwth = re.sub(r'\s\s+?', '', syn_mwth)             # убираем больше двух пробелов
        syn_mwth = re.sub(r'synonyms for ', '', syn_mwth)      # "чистим" начало строки
        syn_mwth = re.sub(item, '', syn_mwth)                  # "чистим" начало строки
        syn_mwth = syn_mwth.rstrip()                           # убираем пробелы в конце стороки
        syn_mwth = re.sub(r'$', ';', syn_mwth)                 # в конце строки ставим ; 
        syn_mwth_concat = syn_mwth_concat + syn_mwth
        syn_mwth_concat = syn_mwth_concat.lstrip()             # убираем пробелы в начале стороки
    syn_header = 'SYNONYMS: ' + url_mwth_link

    for history_ety in soup_ety.find_all('section', class_='word__defination--2q7ZH'):
        history_ety = history_ety.text.lower()
        history_ety_concat = history_ety_concat + history_ety
    history_header = 'HISTORY: ' + url_ety_link
    
    with open('word_info.txt', 'a', encoding = 'utf-8') as word_info:
        word_info.write('\n')
        word_info.write('\n')
        word_info.write(main_header)
        word_info.write('\n')
        word_info.write(translation_header)        
        word_info.write('\n')
        word_info.write(str(ru_words_mtran_list))        
        word_info.write('\n')        
        word_info.write(definition_header)        
        word_info.write('\n')        
        word_info.write(definition_mwd_concat)
        word_info.write('\n')
        word_info.write(syn_header)
        word_info.write('\n')
        word_info.write(syn_mwth_concat)
        word_info.write('\n')
        word_info.write(history_header)
        word_info.write('\n')
        word_info.write(history_ety_concat)

        
#     print(main_header)
#     print(translation_header)
#     print('sub', '\n', ru_words_mtran_list) 
#     print(definition_header)
#     print(definition_mwd_concat)  
#     print(syn_header )
#     print(syn_mwth_concat)
#     print(history_header)
#     print(history_ety_concat)

with open('word_info.txt', 'r', encoding = 'utf-8') as word_info:
    print(word_info.read())

Word list

1 English word: LANKY
TRANSLATION: https://www.multitran.com/m.exe?l1=1&l2=2&s=lanky&langlist=2
['длинный', 'долговязый', 'сухой', 'тонкий', 'тощий и длинноногий']
DEFINITION: https://www.merriam-webster.com/dictionary/lanky
ungracefully tall and thin; tall and thin with usually an awkward quality; very tall and thin; 
SYNONYMS: https://www.merriam-webster.com/thesaurus/lanky
gangling, gangly, rangy, spindling, spindly;
HISTORY: https://www.etymonline.com/word/lanky#etymonline_v_57
1630s, "straight and flat," used of hair, from lank (adj.) + -y (2). sense of "awkwardly tall and thin" is first recorded 1818. fowler writes that "the short form is almost only literary, the long chiefly colloquial." related: lankily (1848); lankiness (1846).

2 English word: SCRAWNY
TRANSLATION: https://www.multitran.com/m.exe?l1=1&l2=2&s=scrawny&langlist=2
['жидковатый', 'костливый', 'костлявый', 'сухопарый', 'тощий', 'тщедушный', 'худой', 'худосочный', 'худощавый', 'цыплячий', 'щуплый']
DEFINI